In [4]:
from langchain_groq import ChatGroq

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
llm = ChatGroq(model="moonshotai/kimi-k2-instruct")

In [7]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 32, 'total_tokens': 40, 'completion_time': 0.013631701, 'prompt_time': 0.001045352, 'queue_time': 0.296447259, 'total_time': 0.014677053}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_c5bd0a648b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cb966ba7-e7fa-4b52-889e-230d70e4b262-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [10]:
result=embeddings.embed_query("What is the capital of France?")

In [11]:
len(result)

768

1.  Data Ingestion

In [3]:
from langchain.document_loaders import PyPDFLoader

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
import os 

In [16]:
file_path = os.path.join(os.getcwd(), "data", "graphmae.pdf")

In [18]:
loader = PyPDFLoader(file_path=file_path)

In [20]:
documents = loader.load()

In [23]:
len(documents)

10

In [24]:
documents[0].page_content[:1000]

'GraphMAE2: A Decoding-Enhanced Masked Self-Supervised\nGraph Learner\nZhenyu Hou\nTsinghua University, China\nhouzy21@mails.tsinghua.edu.cn\nYufei He∗\nBeijing Institute of Technology, China\nyufei.he@bit.edu.cn\nYukuo Cen\nTsinghua University, China\ncyk20@mails.tsinghua.edu.cn\nXiao Liu\nTsinghua University, China\nliuxiao21@mails.tsinghua.edu.cn\nYuxiao Dong†\nTsinghua University, China\nyuxiaod@tsinghua.edu.cn\nEvgeny Kharlamov\nBosch Center for Artificial\nIntelligence, Germany\nevgeny.kharlamov@de.bosch.com\nJie Tang†\nTsinghua University, China\njietang@tsinghua.edu.cn\nABSTRACT\nGraph self-supervised learning (SSL), including contrastive and\ngenerative approaches, offers great potential to address the funda-\nmental challenge of label scarcity in real-world graph data. Among\nboth sets of graph SSL techniques, the masked graph autoencoders\n(e.g., GraphMAE)—one type of generative methods—have recently\nproduced promising results. The idea behind this is to reconstruct\nthe no

In [40]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=250,
    length_function=len
    )

In [41]:
docs = text_splitter.split_documents(documents)

In [42]:
len(docs)

95

In [47]:
docs[0].metadata

{'producer': 'pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'creator': 'LaTeX with acmart 2022/04/09 v1.84 Typesetting articles for the Association for Computing Machinery and hyperref 2020-05-15 v7.00e Hypertext links for LaTeX',
 'creationdate': '2023-04-12T00:29:18+00:00',
 'moddate': '2023-04-12T00:29:18+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'subject': '-  Computing methodologies  ->  Learning latent representations.-  Information systems  ->  Data mining.',
 'title': 'GraphMAE2: A Decoding-Enhanced Masked Self-Supervised Graph Learner',
 'trapped': '/False',
 'source': 'b:\\LLOPS\\document_portal\\notebook\\data\\graphmae.pdf',
 'total_pages': 10,
 'page': 0,
 'page_label': '1'}

In [45]:
docs[0].page_content

'GraphMAE2: A Decoding-Enhanced Masked Self-Supervised\nGraph Learner\nZhenyu Hou\nTsinghua University, China\nhouzy21@mails.tsinghua.edu.cn\nYufei He∗\nBeijing Institute of Technology, China\nyufei.he@bit.edu.cn\nYukuo Cen\nTsinghua University, China\ncyk20@mails.tsinghua.edu.cn\nXiao Liu\nTsinghua University, China\nliuxiao21@mails.tsinghua.edu.cn\nYuxiao Dong†\nTsinghua University, China\nyuxiaod@tsinghua.edu.cn\nEvgeny Kharlamov\nBosch Center for Artificial\nIntelligence, Germany\nevgeny.kharlamov@de.bosch.com\nJie Tang†\nTsinghua University, China\njietang@tsinghua.edu.cn\nABSTRACT\nGraph self-supervised learning (SSL), including contrastive and\ngenerative approaches, offers great potential to address the funda-\nmental challenge of label scarcity in real-world graph data. Among\nboth sets of graph SSL techniques, the masked graph autoencoders\n(e.g., GraphMAE)—one type of generative methods—have recently\nproduced promising results. The idea behind this is to reconstruct'

In [46]:
docs[1].page_content

'both sets of graph SSL techniques, the masked graph autoencoders\n(e.g., GraphMAE)—one type of generative methods—have recently\nproduced promising results. The idea behind this is to reconstruct\nthe node features (or structures)—that are randomly masked from\nthe input—with the autoencoder architecture. However, the per-\nformance of masked feature reconstruction naturally relies on the\ndiscriminability of the input features and is usually vulnerable to\ndisturbance in the features. In this paper, we present a masked\nself-supervised learning framework1 GraphMAE2 with the goal\nof overcoming this issue. The idea is to impose regularization on\nfeature reconstruction for graph SSL. Specifically, we design the\nstrategies of multi-view random re-mask decoding and latent rep-\nresentation prediction to regularize the feature reconstruction. The\nmulti-view random re-mask decoding is to introduce randomness\ninto reconstruction in the feature space, while the latent represen-'

In [48]:
from langchain.vectorstores import FAISS

In [50]:
doc_vector = embeddings.embed_documents([doc.page_content for doc in docs])

In [53]:
len(doc_vector), len(doc_vector[0])

(95, 768)

In [55]:
vector_store = FAISS.from_documents(docs, embeddings)

Token → Word

A chunk is a group of tokens (words/characters) used for indexing and retrieval.

1. Vector stores in RAG can be categorized by storage type:

1. In-memory: Fast, stored in RAM (e.g., FAISS, Chroma)

2. On-disk: Persisted locally (e.g., FAISS with persistence, Chroma)

3. Cloud-based: Scalable, managed solutions (e.g., Pinecone, Weaviate, Milvus, MongoDB Atlas Vector Search, AstraDB)

Note: FAISS has no official cloud-native variant.

2. Data retrieval

In [56]:
relevant_docs = vector_store.similarity_search("What is graph masked autoencoder?", k=2)

In [59]:
relevant_docs[0].page_content, relevant_docs[1].page_content

('Denoting the hidden embedding 𝑯 ∈R𝑁×𝑑, the general goal of\ngraph autoencoders is to learn representation𝑯 or a well-initialized\n𝑓𝐸 through reconstructing input node features or structure:\n𝑯 = 𝑓𝐸(𝑨,𝑿), eG= 𝑓𝐷(𝑨,𝑯) (1)\nwhere eGdenotes the reconstructed graph characteristics, which\ncan be structure, node features or both.\nOverview of masked feature reconstruction. The idea of masked\nautoencoder has seen successful practice in graph SSL [18]. As a\nform of more general denoising autoencoders, it removes a portion\nof data in the graph, e.g., node features or links, with the masking\noperation and learns to predict the masked content. And it has\nbeen demonstrated that reconstructing masked node features as',
 'target generator are discarded, and only the GNN encoder is used\nto generating embeddings or finetuned for downstream tasks.\nExtending to large-scale graph Extending self-supervised learn-\ning to large-scale graphs is of great practical significance, yet few\nefforts have

In [71]:
relevant_doc=vector_store.similarity_search("maksed autoencoder",k=1)

In [72]:
for doc in relevant_doc:
    print(doc.page_content)

the code. The sources of the codes used are as follows:
•BRGL: https://github.com/Namkyeong/BGRL_Pytorch
•GRACE: https://github.com/CRIPAC-DIG/GRACE
•CCA-SSG: https://github.com/hengruizhang98/CCA-SSG/
•GraphMAE:https://github.com/THUDM/GraphMAE


## Keyword filtering

In [73]:
retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 2
        }
)

In [77]:
result = retriever.invoke("What is graph masked autoencoder?")
result[0].page_content

'Denoting the hidden embedding 𝑯 ∈R𝑁×𝑑, the general goal of\ngraph autoencoders is to learn representation𝑯 or a well-initialized\n𝑓𝐸 through reconstructing input node features or structure:\n𝑯 = 𝑓𝐸(𝑨,𝑿), eG= 𝑓𝐷(𝑨,𝑯) (1)\nwhere eGdenotes the reconstructed graph characteristics, which\ncan be structure, node features or both.\nOverview of masked feature reconstruction. The idea of masked\nautoencoder has seen successful practice in graph SSL [18]. As a\nform of more general denoising autoencoders, it removes a portion\nof data in the graph, e.g., node features or links, with the masking\noperation and learns to predict the masked content. And it has\nbeen demonstrated that reconstructing masked node features as'

## Context based retrieval from vector store

In [79]:
prompt_template = """
                  Answer the question based on the context provided below.
                  If the context does not contain enough information to answer the question,
                say "I don't have enough information about this".

                Context: {context}
                Question: {question}

                Answer:
                """


In [80]:
from langchain.prompts import PromptTemplate

In [81]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [82]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n                  Answer the question based on the context provided below.\n                  If the context does not contain enough information to answer the question,\n                say "I don\'t have enough information about this".\n\n                Context: {context}\n                Question: {question}\n\n                Answer:\n                ')

In [84]:
from langchain_core.output_parsers import StrOutputParser

In [85]:
parser = StrOutputParser()

In [86]:
def format_output(docs):

    return "\n\n".join([doc.page_content for doc in docs])

In [89]:
from langchain_core.runnables import RunnablePassthrough

In [90]:
rag_chain = (
    {"context": retriever | format_output, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [91]:
rag_chain.invoke("What is graph masked autoencoder?")

'A graph masked autoencoder is a type of graph autoencoder that follows the masked-autoencoder paradigm used in self-supervised learning. It randomly masks (removes) a portion of the graph’s data—such as node features or links—then trains an encoder–decoder pair to reconstruct the masked content. After pre-training, the decoder is discarded and only the GNN encoder is retained to produce node embeddings or to be fine-tuned for downstream tasks.'

In [93]:
rag_chain.invoke("Can you tell me about treding graph representational learning methods?")

'Based on the provided context, the trending graph representational learning methods are centered around **self-supervised learning (SSL)** approaches for graph data, particularly **graph contrastive learning (GCL)**. Key developments include:\n\n1. **SimGRACE** [47]: A framework that enables graph contrastive learning **without data augmentation**, simplifying the process while maintaining effectiveness.  \n2. **InfoGCL** [48]: Focuses on **information-aware contrastive learning**, incorporating mutual information maximization for better graph representations.  \n3. **GraphCL** [51]: Introduces **augmentation-based contrastive learning**, leveraging transformations like node/feature perturbations to create positive/negative pairs for training.  \n\nThese methods aim to learn robust node/graph embeddings **without labeled data**, with applications in large-scale graphs. However, the context highlights that **scaling SSL to large graphs remains underexplored**.'